# Notebook to deploy SEA LLM 7B

In [2]:
!pip install -U sagemaker transformers --quiet

In [3]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

In [5]:
# Hub Model configuration. https://huggingface.co/models
trust_remote_code=True
hub = {
	'HF_MODEL_ID':'SeaLLMs/SeaLLM-7B-v2.5',
#    'HF_MODEL_TRUST_REMOTE_CODE':json.dumps(trust_remote_code),
	'SM_NUM_GPUS': json.dumps(4)
}

In [6]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="1.4.0"),
	env=hub,
	role=role,
    transformers_version="4.34.1",
)

In [7]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.12xlarge",
	container_startup_health_check_timeout=300,
  )

------------------*

UnexpectedStatusException: Error hosting endpoint huggingface-pytorch-tgi-inference-2024-07-23-00-58-14-326: Failed. Reason: The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint.. Try changing the instance type or reference the troubleshooting page https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference-troubleshooting.html

In [ ]:
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained("SeaLLMs/SeaLLM-7B-v2")

# Prompt to generate
messages = [
    { "role": "user", "content": "Pikirkan tahap demi tahap dan jawab dengan bahasa Indonesia. Bila umur Budi ddua kali umur Wati dan jumlah umur mereka adalah dua belas tahun, berapakah umur Budi ?" },
]

# Generation arguments
payload = {
    "do_sample": True,
    "top_p": 0.6,
    "temperature": 0.1,
    "top_k": 50,
    "max_new_tokens": 1024,
    "repetition_penalty": 1.03,
    "return_full_text": False,
    "stop": ["</s>"]
}

In [ ]:
# send request
chat = predictor.predict({
  "inputs":tok.apply_chat_template(messages,tokenize=False,add_generation_prompt=True), # convert messages to model input
  "parameters":payload
})

print(chat[0]["generated_text"])

In [ ]:
# Cleanup
predictor.delete_model()
predictor.delete_endpoint()
